In [1]:
#Importing the Excel file into Python

import pandas as pd
import numpy as np
import statsmodels.api as sm

# Replace 'your_file.csv' with the path to your actual CSV file
data = pd.read_excel("G:/My Drive/Ireland - Jasleen/Trinity/Coursework/Dissertation/Drafts/Analysis and Findings/Country Wise Analysis/KOSPI 200 CleanData.xlsx")

# Check for missing values and remove rows with missing returns
data = data.dropna(subset=['Return'])

# Display the first few rows of the dataframe
print(data.head(20))

         Date  Unnamed: 1 Day of the week  PX_LAST    Return  \
0  2016-03-21        2005          Monday   244.28 -0.143176   
1  2016-03-22        2004         Tuesday   245.41  0.461517   
2  2016-03-23        2003       Wednesday   245.23 -0.073374   
3  2016-03-24        2002        Thursday   244.08 -0.470051   
4  2016-03-25        2001          Friday   243.79 -0.118884   
5  2016-03-28        2000          Monday   243.95  0.065609   
6  2016-03-29        1999         Tuesday   245.53  0.645585   
7  2016-03-30        1998       Wednesday   246.54  0.410511   
8  2016-03-31        1997        Thursday   245.86 -0.276198   
9  2016-04-01        1996          Friday   242.36 -1.433804   
10 2016-04-04        1995          Monday   243.05  0.284296   
11 2016-04-05        1994         Tuesday   240.58 -1.021451   
12 2016-04-06        1993       Wednesday   241.99  0.584373   
13 2016-04-07        1992        Thursday   242.29  0.123895   
14 2016-04-08        1991          Frida

In [2]:
# Defining data range
# Specify the start and end dates of the desired range
start_date = '2016-05-02'
end_date = '2024-05-10'

# Filter the dataframe based on the date range
data = data.loc[(data['Date'] >= start_date) & (data['Date'] <= end_date)]

# Print the selected columns
print(data)

           Date  Unnamed: 1 Day of the week  PX_LAST    Return  \
25   2016-05-09        1973          Monday   242.23 -0.444866   
26   2016-05-10        1972         Tuesday   243.68  0.596820   
27   2016-05-11        1971       Wednesday   243.07 -0.250642   
28   2016-05-12        1970        Thursday   242.34 -0.300777   
29   2016-05-13        1969          Friday   240.60 -0.720590   
...         ...         ...             ...      ...       ...   
1670 2024-04-22          15          Monday   356.89  1.215006   
1671 2024-04-23          14         Tuesday   355.98 -0.255306   
1672 2024-04-24          13       Wednesday   363.60  2.117981   
1673 2024-04-25          12        Thursday   356.51 -1.969207   
1674 2024-04-26          11          Friday   361.02  1.257107   

                       Week Day of Week  
25    2016-05-09/2016-05-15      Monday  
26    2016-05-09/2016-05-15     Tuesday  
27    2016-05-09/2016-05-15   Wednesday  
28    2016-05-09/2016-05-15    Thursday

In [3]:
# Drop rows with NaT in the 'Date' column
data = data.dropna(subset=['Date'])

# Extract the day of the week from the 'Date' column
data['Day_of_Week'] = data['Date'].dt.day_name()

# Perform one-hot encoding to create dummy variables for each day of the week
data_with_dummies = pd.get_dummies(data, columns=['Day_of_Week'], drop_first=False)

# Check if any of the columns contain boolean values and convert them to integer (0 and 1)
for col in data_with_dummies.columns:
    if data_with_dummies[col].dtype == 'bool':
        data_with_dummies[col] = data_with_dummies[col].astype(int)


# Display the resulting dataframe
data_with_dummies.head()

,Date,Unnamed: 1,Day of the week,PX_LAST,Return,Week,Day of Week,Day_of_Week_Friday,Day_of_Week_Monday,Day_of_Week_Thursday,Day_of_Week_Tuesday,Day_of_Week_Wednesday
25,2016-05-09,1973,Monday,242.23,-0.444866,2016-05-09/2016-05-15,Monday,0,1,0,0,0
26,2016-05-10,1972,Tuesday,243.68,0.596820,2016-05-09/2016-05-15,Tuesday,0,0,0,1,0
27,2016-05-11,1971,Wednesday,243.07,-0.250642,2016-05-09/2016-05-15,Wednesday,0,0,0,0,1
28,2016-05-12,1970,Thursday,242.34,-0.300777,2016-05-09/2016-05-15,Thursday,0,0,1,0,0
29,2016-05-13,1969,Friday,240.60,-0.720590,2016-05-09/2016-05-15,Friday,1,0,0,0,0


In [4]:
# Sort by the 'Return' column in descending order
sorted_df = data_with_dummies.sort_values(by='Return', ascending=False)
print(sorted_df.head(10))

           Date  Unnamed: 1 Day of the week  PX_LAST    Return  \
826  2020-03-24        1021         Tuesday   220.34  8.754788   
824  2020-03-20        1023          Friday   213.67  6.972190   
827  2020-03-25        1020       Wednesday   232.89  5.539443   
871  2020-06-16         965         Tuesday   282.59  5.319669   
1580 2023-11-06         128          Monday   334.16  4.881411   
994  2021-01-08         825          Friday   430.22  4.616589   
835  2020-04-06        1012          Monday   240.81  3.856481   
1374 2022-11-11         370          Friday   323.09  3.772009   
1023 2021-02-25         793        Thursday   422.52  3.703055   
539  2018-11-02        1363          Friday   272.56  3.589487   

                       Week Day of Week  Day_of_Week_Friday  \
826   2020-03-23/2020-03-29     Tuesday                   0   
824   2020-03-16/2020-03-22      Friday                   1   
827   2020-03-23/2020-03-29   Wednesday                   0   
871   2020-06-15/2020

In [5]:
# Calculating the indices / rows for each

total_rows = len(sorted_df)
top_20_percent_index = int(0.2 * total_rows)
bottom_20_percent_index = int(0.8 * total_rows)

In [6]:
# Top 20%
top_20_df = sorted_df.head(top_20_percent_index)

# Bottom 20%
bottom_20_df = sorted_df.tail(total_rows - bottom_20_percent_index)

# Middle 60%
middle_60_df = sorted_df.iloc[top_20_percent_index:bottom_20_percent_index]

In [7]:
# Top 20% Stats

from statsmodels.formula.api import ols
import statsmodels.api as sm

# Define the independent variables (dummy variables for days of the week)
X_6 = top_20_df[['Day_of_Week_Monday', 
                       'Day_of_Week_Thursday', 'Day_of_Week_Tuesday', 
                       'Day_of_Week_Wednesday','Day_of_Week_Friday']]

# Add a constant to the independent variables
# X_6 = sm.add_constant(X_6)

# Define the dependent variable (Returns)
Y_6 = top_20_df['Return']
# Check data types
# print(data_with_dummies.dtypes)

# Create the ANOVA model
ANOVAmodel = ols('Y_6~X_6', data=top_20_df).fit()

# Perform ANOVA
anova_table = sm.stats.anova_lm(ANOVAmodel)

# Print the ANOVA table
print(anova_table)

# Perform OLS regression
model6 = sm.OLS(Y_6, X_6).fit()

# Print the summary of the regression
print(model6.summary())

             df      sum_sq   mean_sq         F    PR(>F)
X_6         5.0    2.952800  0.590560  0.818538  0.537123
Residual  325.0  234.481546  0.721482       NaN       NaN
                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.3497
Date:                Wed, 19 Jun 2024   Prob (F-statistic):              0.844
Time:                        11:20:55   Log-Likelihood:                -411.87
No. Observations:                 330   AIC:                             833.7
Df Residuals:                     325   BIC:                             852.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    

In [8]:
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey, het_arch
# Add a constant column to the exog variable
exog_with_constant = sm.add_constant(model6.model.exog)

# Perform the Breusch-Pagan test with the updated exog variable
bp_test = het_breuschpagan(model6.resid, exog_with_constant)
bp_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Pagan test:", dict(zip(bp_labels, bp_test)))

# Breusch-Godfrey test for autocorrelation
bg_test = acorr_breusch_godfrey(model6, nlags=1)
bg_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Godfrey test:", dict(zip(bg_labels, bg_test)))


Breusch-Pagan test: {'Lagrange multiplier statistic': 2.2756798005355927, 'p-value': 0.8098321483701652, 'f-value': 0.5641906089880059, 'f p-value': 0.6888149894658284}
Breusch-Godfrey test: {'Lagrange multiplier statistic': 250.61426447919763, 'p-value': 1.9077661081529734e-56, 'f-value': 1022.8414608564353, 'f p-value': 2.9153312486839195e-102}


In [9]:
# Newey-West standard errors - Corrects for autocorrelation
nw_summary = model6.get_robustcov_results(cov_type='HAC', maxlags=1)
print(nw_summary.summary())

                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 19 Jun 2024   Prob (F-statistic):                nan
Time:                        11:20:55   Log-Likelihood:                -411.87
No. Observations:                 330   AIC:                             833.7
Df Residuals:                     325   BIC:                             852.7
Df Model:                           4                                         
Covariance Type:                  HAC                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Day_of_Week_Monday        1.45

In [10]:
# Middle 60% Stats

import statsmodels.api as sm

# Define the independent variables (dummy variables for days of the week)
X_7 = middle_60_df[['Day_of_Week_Monday', 
                       'Day_of_Week_Thursday', 'Day_of_Week_Tuesday', 
                       'Day_of_Week_Wednesday','Day_of_Week_Friday']]

# Add a constant to the independent variables
# X_7 = sm.add_constant(X_7)

# Define the dependent variable (Returns)
Y_7 = middle_60_df['Return']
# Check data types
# print(data_with_dummies.dtypes)

# Create the ANOVA model
ANOVAmodel = ols('Y_7~X_7', data=middle_60_df).fit()

# Perform ANOVA
anova_table = sm.stats.anova_lm(ANOVAmodel)

# Print the ANOVA table
print(anova_table)

# Perform OLS regression
model7 = sm.OLS(Y_7, X_7).fit()

# Print the summary of the regression
print(model7.summary())


             df      sum_sq   mean_sq         F    PR(>F)
X_7         5.0    0.815839  0.163168  1.133041  0.340977
Residual  985.0  141.848668  0.144009       NaN       NaN
                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.407
Date:                Wed, 19 Jun 2024   Prob (F-statistic):              0.230
Time:                        11:20:55   Log-Likelihood:                -442.99
No. Observations:                 990   AIC:                             896.0
Df Residuals:                     985   BIC:                             920.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    

In [11]:
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey, het_arch
# Add a constant column to the exog variable
exog_with_constant = sm.add_constant(model7.model.exog)

# Perform the Breusch-Pagan test with the updated exog variable
bp_test = het_breuschpagan(model7.resid, exog_with_constant)
bp_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Pagan test:", dict(zip(bp_labels, bp_test)))

# Breusch-Godfrey test for autocorrelation
bg_test = acorr_breusch_godfrey(model7, nlags=1)
bg_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Godfrey test:", dict(zip(bg_labels, bg_test)))


Breusch-Pagan test: {'Lagrange multiplier statistic': 3.818158417376809, 'p-value': 0.575882499079183, 'f-value': 0.9533956828592212, 'f p-value': 0.432318398929165}
Breusch-Godfrey test: {'Lagrange multiplier statistic': 980.7124033662706, 'p-value': 2.797273666905597e-215, 'f-value': 103904.27609741152, 'f p-value': 0.0}


In [12]:
# Newey-West standard errors - Corrects for autocorrelation
nw_summary = model7.get_robustcov_results(cov_type='HAC', maxlags=1)
print(nw_summary.summary())

                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 19 Jun 2024   Prob (F-statistic):                nan
Time:                        11:20:55   Log-Likelihood:                -442.99
No. Observations:                 990   AIC:                             896.0
Df Residuals:                     985   BIC:                             920.5
Df Model:                           4                                         
Covariance Type:                  HAC                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Day_of_Week_Monday        0.05

In [13]:
# Bottom 20% Stats

import statsmodels.api as sm

# Define the independent variables (dummy variables for days of the week)
X_8 = bottom_20_df[['Day_of_Week_Monday', 
                       'Day_of_Week_Thursday', 'Day_of_Week_Tuesday', 
                       'Day_of_Week_Wednesday','Day_of_Week_Friday']]

# Add a constant to the independent variables
# X_8 = sm.add_constant(X_8)

# Define the dependent variable (Returns)
Y_8 = bottom_20_df['Return']
# Check data types
# print(data_with_dummies.dtypes)

# Create the ANOVA model
ANOVAmodel = ols('Y_8~X_8', data=bottom_20_df).fit()

# Perform ANOVA
anova_table = sm.stats.anova_lm(ANOVAmodel)

# Print the ANOVA table
print(anova_table)

# Perform OLS regression
model8 = sm.OLS(Y_8, X_8).fit()

# Print the summary of the regression
print(model8.summary())

             df      sum_sq   mean_sq         F    PR(>F)
X_8         5.0    2.636898  0.527380  0.741978  0.592493
Residual  325.0  231.002072  0.710776       NaN       NaN
                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.5337
Date:                Wed, 19 Jun 2024   Prob (F-statistic):              0.711
Time:                        11:20:55   Log-Likelihood:                -409.40
No. Observations:                 330   AIC:                             828.8
Df Residuals:                     325   BIC:                             847.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    

In [14]:
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey, het_arch
# Add a constant column to the exog variable
exog_with_constant = sm.add_constant(model8.model.exog)

# Perform the Breusch-Pagan test with the updated exog variable
bp_test = het_breuschpagan(model8.resid, exog_with_constant)
bp_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Pagan test:", dict(zip(bp_labels, bp_test)))

# Breusch-Godfrey test for autocorrelation
bg_test = acorr_breusch_godfrey(model8, nlags=1)
bg_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Godfrey test:", dict(zip(bg_labels, bg_test)))


Breusch-Pagan test: {'Lagrange multiplier statistic': 4.091216939235961, 'p-value': 0.5363591715848334, 'f-value': 1.0199521878210445, 'f p-value': 0.39703339727857845}
Breusch-Godfrey test: {'Lagrange multiplier statistic': 319.50891137219526, 'p-value': 1.8529821188744e-71, 'f-value': 9867.506696133303, 'f p-value': 1.0640842218515469e-244}


In [15]:
# Newey-West standard errors - Corrects for autocorrelation
nw_summary = model8.get_robustcov_results(cov_type='HAC', maxlags=1)
print(nw_summary.summary())

                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 19 Jun 2024   Prob (F-statistic):                nan
Time:                        11:20:55   Log-Likelihood:                -409.40
No. Observations:                 330   AIC:                             828.8
Df Residuals:                     325   BIC:                             847.8
Df Model:                           4                                         
Covariance Type:                  HAC                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Day_of_Week_Monday       -1.54